In [ ]:
import os
import cv2
import shutil
import math
from pathlib import Path
import PIL
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from skimage.transform import resize   # for resizing imagesy
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from tensorflow.keras import layers, models, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense, Dropout,Input, Add, ReLU, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
count = 0
videoFile = "../input/cricket-final/Misc (1).avi"
cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

In [ ]:
lst = list()
for i in range(0,count):
    lst.append('frame'+str(i)+'.jpg')
df = pd.DataFrame(lst)
df.columns =['Image_ID']

In [ ]:
import os
os.makedirs(os.path.join('./','CricketTest'))

In [ ]:
import shutil
for i in df['Image_ID']:    
    get_image = os.path.join('.//',i)
    move_image_to_cat = shutil.move(get_image, './CricketTest/' + str(i))

In [ ]:
model_4 = keras.models.load_model('../input/Inception-V3-50-epochh5/Inception V3 50_epoch.h5')

In [ ]:
# Generate Test Data
test_list = tf.io.gfile.listdir('./CricketTest/')
test_data = []
for f in test_list:
    img = tf.keras.preprocessing.image.load_img('./CricketTest/' + f, color_mode = "rgb", target_size = (227,227))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img/255
    test_data.append(img)

In [ ]:
pred_new = model_4.predict(tf.convert_to_tensor(test_data))
prednew = np.argmax(pred_new, axis = 1)
prednew

In [ ]:
df['Class'] = prednew

In [ ]:
Batting = df[df['Class']==0]
Bowling = df[df['Class']==1]
Boundary = df[df['Class']==2]
CloseUp = df[df['Class']==3]
Misc = df[df['Class']==4]

In [ ]:
import numpy as np
import glob
 
img_array = []
for i,row in Batting.iterrows():
    filename = os.path.join('./CricketTest/',row.Image_ID)
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('Batting.avi',cv2.VideoWriter_fourcc(*'DIVX'), 1, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()